# 06-build-model-TextCatBERT

## Main objectives:

- Use spaCy's BERT architecture to train a BERT model.
    - [https://github.com/explosion/spacy-transformers](https://github.com/explosion/spacy-transformers)
- Use spaCy generated docs to train this model
- Run basic validation and evaluation 

Ended up using a much smaller train and dev dataset (100 vs. 5000 used in other models) due to lack of GPU for faster training

In [1]:
import spacy
# load an english language model in spacy
nlp = spacy.load("en_core_web_lg")

In [2]:
!ls data

dev.spacy  train.spacy


# Validate configuration file

In [11]:
# validate configuration
!python -m spacy debug config ./config/config-TextCatBERT.cfg

python-exec: Invalid impl in /etc/python-exec/python-exec.conf: python3.6
python-exec: Invalid impl in /etc/python-exec/python-exec.conf: python2.7
2021-03-25 20:50:39.832972: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0

============================= Config validation =============================

===================== Config validation for [initialize] =====================

====================== Config validation for [training] ======================
✔ Config is valid


In [12]:
!python -m spacy debug data ./config/config-TextCatBERT.cfg

python-exec: Invalid impl in /etc/python-exec/python-exec.conf: python3.6
python-exec: Invalid impl in /etc/python-exec/python-exec.conf: python2.7
2021-03-25 20:50:44.814028: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0

============================ Data file validation ============================
✔ Corpus is loadable
✔ Pipeline can be initialized with data

=============================== Training stats ===============================
Language: en
Training pipeline: transformer, textcat
100 training docs
100 evaluation docs
✔ No overlap between training and evaluation data
⚠ Low number of examples to train a new pipeline (100)

============================== Vocab & Vectors ==============================
ℹ 5739 total word(s) in the data (1931 unique)
ℹ No word vectors present in the package

================================== Summary ==================================
✔ 3 checks passed
⚠ 1 warning


# Train the model

In [ ]:
%env TOKENIZERS_PARALLELISM=false
!python -m spacy train ./config/config-TextCatBERT.cfg --output ./models/textCatBERT


env: TOKENIZERS_PARALLELISM=false
python-exec: Invalid impl in /etc/python-exec/python-exec.conf: python3.6
python-exec: Invalid impl in /etc/python-exec/python-exec.conf: python2.7
2021-03-25 20:51:32.029232: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2021-03-25 20:51:33,785] [INFO] Set up nlp object from config
[2021-03-25 20:51:33,918] [INFO] Pipeline: ['transformer', 'textcat']
[2021-03-25 20:51:33,922] [INFO] Created vocabulary
[2021-03-25 20:51:33,922] [INFO] Finished initializing nlp object
[2021-03-25 20:52:06,935] [INFO] Initialized pipeline components: ['transformer', 'textcat']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['transformer', 'textcat']
ℹ Initial learn rate: 0.0
wandb: W&B API key is configured (use `wandb login --relogin` to force

# Evaluate our best model output and save metrics to disk

For hyperparameter tuning, experimented with different BOW attributes and TexCatEnsemble parameters:

- ngram_size = 4 (TODO: should try again with ngram_size = 2 after doign BOW analysis)
- adjusted width to 128, nominal performance gain
- tok2vec model embed attributed modified to include:
    - "ORTH", "LOWER", "NORM", "PREFIX", "SUFFIX", "SHAPE", "ID"
    - [2000, 2000, 1000, 1000, 1000, 1000]

Also adjusted the width size from 64 to 96, which only resulted in a nominal increase in performance.

Training was tested on training datasets of 100, 500, 1000 and finally, 5000.

In [9]:
!python -m spacy evaluate ./models/textCatBERT/model-best ./data/dev.spacy --output ./evaluate/model-textCatBERT-metrics.json

python-exec: Invalid impl in /etc/python-exec/python-exec.conf: python3.6
python-exec: Invalid impl in /etc/python-exec/python-exec.conf: python2.7
2021-03-25 20:36:33.789066: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
ℹ Using CPU

================================== Results ==================================

TOK                   100.00
TEXTCAT (macro AUC)   92.85 
SPEED                 34783 


=========================== Textcat F (per label) ===========================

               P       R       F
positive   87.52   81.36   84.33
negative   82.59   88.40   85.39


======================== Textcat ROC AUC (per label) ========================

           ROC AUC
positive      0.93
negative      0.93

✔ Saved results to evaluate/model-textCatEnsemble-metrics.json
